## Pré-processamento

In [1]:
!hostname

k003-002.hpcfund


In [2]:
import os

WORK_DIR = os.getenv('WORK')

DATA_FOLDER = os.path.join(WORK_DIR, "data")

CACHED_DATA_FOLDER = os.path.join(WORK_DIR, "cached_data")

# Salvamos o path do Cache par ao HuggingFace
os.environ['HF_HOME'] = CACHED_DATA_FOLDER

CACHED_DATA_FOLDER

'/work1/lgarcia/renneruan/cached_data'

In [ ]:
# !pip install nltk

In [3]:
from datasets import load_from_disk
import nltk

from multiprocessing import cpu_count



In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home1/renneruan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
os.chdir(WORK_DIR)
print(os.getcwd())

/work1/lgarcia/renneruan


### Tratamento


In [6]:
concatenated_save_path = os.path.join(DATA_FOLDER, 'concatenated_raw_datasets')

In [7]:
concatenated_datasets = load_from_disk(concatenated_save_path)
print("Successfully reloaded dataset from disk:")
print(concatenated_datasets)

Loading dataset from disk:   0%|          | 0/41 [00:00<?, ?it/s]

Successfully reloaded dataset from disk:
Dataset({
    features: ['text'],
    num_rows: 4643042
})


In [8]:
stopwords = nltk.corpus.stopwords.words('portuguese')

In [9]:
def analize_texts(row):

    list_paragraph = []
    list_words = []
    list_stopwords = []
    list_average = []

    for doc in row["text"]:
        for paragraph in doc.split('\n'):

            # strip whitespaces
            paragraph = paragraph.strip()

            # skip single or empty worded paragraphs
            if (len(paragraph.split()) < 2):
                continue

            # count how many stopwords are in the paragraph
            stopwords_cnt = 0    
            for word in paragraph.split():
                for stop in stopwords:
                    if stop.casefold() == word.casefold():  # insensitive case
                        stopwords_cnt += 1
                        break # count once and speed up everything
            
            # count non whitespace characters
            characters = 0
            for word in paragraph.split():
                characters += len(word)

            list_paragraph.append(paragraph)
            list_words.append(len(paragraph.split()))
            list_stopwords.append(stopwords_cnt)
            list_average.append(characters/len(paragraph.split()))

    return {
        "paragraphs" : list_paragraph,
        "num_words" : list_words,
        "stopwords" : list_stopwords,
        "average" : list_average
    }

In [10]:
preprocessed_datasets = concatenated_datasets.map(
    analize_texts,
    batched = True,
    remove_columns=["text"],
    num_proc = cpu_count(),
)

Map (num_proc=128):   0%|          | 0/4643042 [00:00<?, ? examples/s]

In [11]:
preprocessed_datasets = preprocessed_datasets.rename_column("paragraphs", "text")
preprocessed_datasets

Dataset({
    features: ['text', 'num_words', 'stopwords', 'average'],
    num_rows: 156783923
})

In [12]:
preprocessed_save_path = os.path.join(DATA_FOLDER, 'preprocessed_datasets')

In [13]:
preprocessed_datasets.save_to_disk(preprocessed_save_path)

Saving the dataset (0/49 shards):   0%|          | 0/156783923 [00:00<?, ? examples/s]

In [14]:
preprocessed_datasets[:10]["text"]

['Astronomia é uma ciência natural que estuda corpos celestes (como estrelas, planetas, cometas, nebulosas, aglomerados de estrelas, galáxias) e fenômenos que se originam fora da atmosfera da Terra (como a radiação cósmica de fundo em micro-ondas). Preocupada com a evolução, a física e a química de objetos celestes, bem como a formação e o desenvolvimento do universo.',
 'A astronomia é uma das mais antigas ciências. Culturas pré-históricas deixaram registrados vários artefatos astronômicos, como Stonehenge, os montes de Newgrange e os menires. As primeiras civilizações, como os babilônios, gregos, chineses, indianos, persas e maias realizaram observações metódicas do céu noturno. No entanto, a invenção do telescópio permitiu o desenvolvimento da astronomia moderna. Historicamente, a astronomia incluiu disciplinas tão diversas como astrometria, navegação astronômica, astronomia observacional e a elaboração de calendários. Durante o período medieval, seu estudo era obrigatório e estava 

In [15]:
preprocessed_datasets[-10:]["text"]

['Antigamente era costume de invocar apenas aulas de santos e, em seguida, foram adicionados nomes individuais, e em muitos lugares santos foram adicionados locais (Rock ", A Igreja de Nossa Padres ", Londres, 1903, 182;" Manuale Lincopense ", Paderborn, 1904, 71).',
 'Para obter uniformidade, modificações e acréscimos para os aprovados foram proibidos (SRC, 2093, 3236, 3313).',
 'Publicação informações escritas por Francis Mershman.',
 'Transcrita por Thomas M. Barrett.',
 'Dedicado aos pobres almas do Purgatório A Enciclopédia Católica, Volume IX.',
 'Publicado 1910.',
 'Posts do dia 3 abril 2012',
 'A partir desta quarta-feira, os leitores de Zero Hora terão um endereço certo na internet para tirar suas dúvidas sobre as eleições deste ano.',
 'Um guia completo trará informações sobre como regularizar o título, regras da propaganda eleitoral e voto biométrico.',
 'Além disso, o eleitor poderá acessar as notícias atualizadas dos candidatos e das campanhas nos municípios gaúchos.']

In [16]:
preprocessed_datasets[:10]["num_words"]

[57, 99, 89, 34, 42, 168, 64, 77, 91, 88]

In [17]:
preprocessed_datasets[-10:]["stopwords"]

[11, 5, 1, 1, 3, 0, 1, 9, 5, 6]

In [18]:
preprocessed_datasets[-10:]["average"]

[5.439024390243903,
 6.333333333333333,
 8.0,
 5.8,
 5.909090909090909,
 7.0,
 3.3333333333333335,
 4.958333333333333,
 5.9411764705882355,
 5.823529411764706]

In [19]:
cleaned_datasets = preprocessed_datasets.filter(
    lambda example: 
            example["num_words"] >= 20 
        and example["num_words"] <= 512
        and example["stopwords"] >= 1
        and example["average"] >= 2 
        and example["average"] <= 15
    )

cleaned_datasets

Filter:   0%|          | 0/156783923 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'num_words', 'stopwords', 'average'],
    num_rows: 63029659
})

In [20]:
cleaned_datasets = cleaned_datasets.remove_columns([col for col in cleaned_datasets.column_names if col != "text"])  # only keep the 'text' column

### Split

In [21]:
split_dataset = cleaned_datasets.train_test_split(test_size=0.1, shuffle=True, seed=42)

split_dataset 

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 56726693
    })
    test: Dataset({
        features: ['text'],
        num_rows: 6302966
    })
})

In [22]:
split_save_path = os.path.join(DATA_FOLDER, 'split_datasets')

In [23]:
split_dataset.save_to_disk(split_save_path)

Saving the dataset (0/15 shards):   0%|          | 0/56726693 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/6302966 [00:00<?, ? examples/s]